In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Vector Embedding Ingestion with Apache Beam and AlloyDB

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/alloydb_product_catalog_embeddings.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/alloydb_product_catalog_embeddings.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>


# Introduction

This Colab demonstrates how to generate embeddings from data and ingest them into [AlloyDB](https://cloud.google.com/alloydb), Google Cloud's fully managed, PostgreSQL-compatible database service. We'll use Apache Beam and Dataflow for scalable data processing.

The goal of this notebook is to make it easy for users to get started with generating embeddings at scale using Apache Beam and storing them in AlloyDB. We focus on building efficient ingestion pipelines that can handle various data sources and embedding models.

## Example: Furniture Product Catalog

We'll work with a sample e-commerce dataset representing a furniture product catalog. Each product has:

*   **Structured fields:** `id`, `name`, `category`, `price`
*   **Detailed text descriptions:** Longer text describing the product's features.
*   **Additional metadata:** `material`, `dimensions`

## Pipeline Overview
We will build a pipeline to:
1. Read product data
2. Convert unstructured product data, to `Chunk`<sup>[1]</sup> type
2. Generate Embeddings: Use a pre-trained Hugging Face model (via MLTransform) to create vector embeddings
3. Write to AlloyDB: Store the embeddings in an AlloyDB vector database

Here's a visualization of the data flow:

| Stage                     | Data Representation                                      | Notes                                                                                                                   |
| :------------------------ | :------------------------------------------------------- | :---------------------------------------------------------------------------------------------------------------------- |
| **1. Ingest Data**      | `{`<br> `  "id": "desk-001",`<br> `  "name": "Modern Desk",`<br> `  "description": "Sleek...",`<br> `  "category": "Desks",`<br> `  ...`<br> `}` | Supports:<br>- Reading from batch (e.g., files, databases)<br>- Streaming sources (e.g., Pub/Sub).                 |
| **2. Convert to Chunks** | `Chunk(` <br>  &nbsp;&nbsp;`id="desk-001",` <br>  &nbsp;&nbsp;`content=Content(` <br>   &nbsp;&nbsp;&nbsp;&nbsp;`text="Modern Desk"` <br> &nbsp;&nbsp; `),` <br>  &nbsp;&nbsp;`metadata={...}` <br> `)`       | - `Chunk` is the structured input for generating and ingesting embeddings.<br>- `chunk.content.text` is the field that is embedded.<br> - Converting to `Chunk` does not mean breaking data into smaller pieces,<br>&nbsp;&nbsp; it's simply organizing your data in a standard format for the embedding pipeline.<br> - `Chunk` allows data to flow seamlessly throughout embedding pipelines. |
| **3. Generate Embeddings**| `Chunk(` <br>  &nbsp;&nbsp;`id="desk-001",`<br>  &nbsp;&nbsp;`embedding=[-0.1, 0.6, ...],`<br>  `...)`  | Supports:<br>- Local Hugging Face models<br>- Remote Vertex AI models<br>- Custom embedding implementations.          |
| **4. Write to AlloyDB** | **AlloyDB Table (Example Row):**<br>`id: desk-001`<br>`embedding: [-0.1, 0.6, ...]`<br> `name = "Modern Desk"`,<br>`Other fields ...` | Supports:<br>- Custom schemas<br>- Conflict resolution strategies for handling updates                               |


[1]: Chunk represents an embeddable unit of input. It specifies which fields should be embedded and which fields should be treated as metadata. Converting to Chunk does not necessarily mean breaking your text into smaller pieces - it's primarily about structuring your data for the embedding pipeline. For very long texts that exceed the embedding model's maximum input size, you can optionally [use Langchain TextSplitters](https://beam.apache.org/releases/pydoc/2.63.0/apache_beam.ml.rag.chunking.langchain.html) to break the text into smaller `Chunk`'s.

## Execution Environments

This notebook demonstrates two execution environments:

1. **DirectRunner (Local Execution)**: All examples in this notebook run on DirectRunner by default, which executes the pipeline locally. This is ideal for development, testing, and processing small datasets.

2. **DataflowRunner (Distributed Execution)**: The [Run on Dataflow](#scrollTo=Quick_Start_Run_on_Dataflow) section demonstrates how to execute the same pipeline on Google Cloud Dataflow for scalable, distributed processing. This is recommended for production workloads and large datasets.

All examples in this notebook can be adapted to run on Dataflow by following the pattern shown in the "Run on Dataflow" section.

# Setup and Prerequisites

This example requires:
1. An AlloyDB instance with pgvector extension enabled
2. Apache Beam 2.63.0 or later

## Install Packages and Dependencies

First, let's install the Python packages required for the embedding and ingestion pipeline:

In [ ]:
# Apache Beam with GCP support
!pip install apache_beam[gcp]==v2.63.0 --quiet
# Huggingface sentence-transformers for embedding models
!pip install sentence-transformers --quiet

Next, let's install psycopg2-binary to help set up our test database.

In [ ]:
!pip install psycopg2-binary --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.0 MB/s eta 0:00:00


## Database Setup

To connect to AlloyDB, you'll need:
1. The JDBC connection URL
2. Database credentials
3. The pgvector extension enabled in your database

Replace these placeholder values with your actual AlloyDB connection details:

In [ ]:
ALLOYDB_HOST = "" # @param {type:'string'}
DB_NAME = "postgres" #  @param {type:'string'}
JDBC_URL = f"jdbc:postgresql://{ALLOYDB_HOST}:5432/{DB_NAME}"
DB_USER = "postgres" # @param {type:'string'}
DB_PASSWORD = "" # @param {type:'string'}

### To connect from Colab to AlloyDB, you'll need one of the following:
1. Public IP: Enable public IP on your AlloyDB instance and add your Colab IP to the authorized networks
2. Auth Proxy: Use the Cloud SQL Auth Proxy to establish a secure connection
3. VPC: Run this notebook on a Compute Engine VM in the same VPC as your AlloyDB instance

Your current IP address (for configuring AlloyDB authorized networks if using public IP):


In [ ]:
!curl ifconfig.me

34.168.233.188

In [ ]:
#@title Postgres helpers for creating tables and verifying data
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def setup_alloydb_table(host: str,
                       database: str,
                      table_name: str,
                       user: str,
                       password: str,
                       port: int = 5432):
    """Set up AlloyDB table with vector extension and proper schema.

    Args:
        host: AlloyDB instance host
        database: Database name
        user: Database user
        password: Database password
        port: Database port (default: 5432)
    """
    # Connection string
    conn_string = f"host={host} dbname={database} user={user} password={password} port={port}"

    try:
        # Connect to the database
        conn = psycopg2.connect(conn_string)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = conn.cursor()

        print("Connected to AlloyDB successfully!")

        # Create pgvector extension if it doesn't exist
        print("Creating pgvector extension...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

        # Create the product_embeddings table
        print("Creating table...")
        cur.execute(f"""
          DROP TABLE IF EXISTS {table_name};
        """)
        cur.execute(f"""
          CREATE TABLE IF NOT EXISTS {table_name} (
              {table_schema}
          );
        """)

        print("Setup completed successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()

# Example usage (user will need to provide their actual connection details)
"""
To set up your AlloyDB table, run the following with your connection details:

setup_alloydb_table(
    host="your-alloydb-host",
    database="your-database",
    user="your-username",
    password="your-password"
)
"""

"""### Test the Connection and Table Setup

You can verify the setup with a simple query:
"""

def test_alloydb_connection(host: str,
                          database: str,
                          table_name: str,
                          user: str,
                          password: str,
                          port: int = 5432):
    """Test the AlloyDB connection and verify table creation.

    Args:
        host: AlloyDB instance host
        database: Database name
        user: Database user
        password: Database password
        port: Database port (default: 5432)
    """
    conn_string = f"host={host} dbname={database} user={user} password={password} port={port}"

    try:
        conn = psycopg2.connect(conn_string)
        cur = conn.cursor()

        # Check if table exists
        cur.execute(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_name = '{table_name}'
            );
        """)
        table_exists = cur.fetchone()[0]

        if table_exists:
            print(f"✓ {table_name} table exists")

            # Check if vector extension is installed
            cur.execute("SELECT * FROM pg_extension WHERE extname = 'vector';")
            if cur.fetchone():
                print("✓ pgvector extension is installed")
            else:
                print("✗ pgvector extension is not installed")
        else:
            print(f"✗ {table_name} table does not exist")

    except Exception as e:
        print(f"Connection test failed: {e}")
    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()

def verify_embeddings(host: str,
                     database: str,
                     table_name: str,
                     user: str,
                     password: str,
                     port: int = 5432):
    """Connect to AlloyDB and print all written products."""
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )

    try:
        with conn.cursor() as cur:
            # Simple SELECT * query
            cur.execute(f"SELECT * FROM {table_name};")
            rows = cur.fetchall()

            # Get column names from cursor description
            columns = [desc[0] for desc in cur.description]

            print(f"\nFound {len(rows)} products:")
            print("-" * 80)

            # Print each row with column names
            for row in rows:
                for col, val in zip(columns, row):
                    print(f"{col}: {val}")
                print("-" * 80)

    finally:
        conn.close()

## Create Sample Product Catalog Data

We'll create a typical e-commerce catalog where you might want to:
- Generate embeddings for product text
- Store vectors alongside product data
- Enable vector similarity features

Example product:
```python
{
    "id": "desk-001",
    "name": "Modern Minimalist Desk",
    "description": "Sleek minimalist desk with clean lines and a spacious work surface. "
                  "Features cable management system and sturdy steel frame. "
                  "Perfect for contemporary home offices and workspaces.",
    "category": "Desks",
    "price": 399.99,
    "material": "Engineered Wood, Steel",
    "dimensions": "60W x 30D x 29H inches"
}
```

In [ ]:
#@title Create sample data
PRODUCTS_DATA = [
    {
        "id": "desk-001",
        "name": "Modern Minimalist Desk",
        "description": "Sleek minimalist desk with clean lines and a spacious work surface. "
                      "Features cable management system and sturdy steel frame. "
                      "Perfect for contemporary home offices and workspaces.",
        "category": "Desks",
        "price": 399.99,
        "material": "Engineered Wood, Steel",
        "dimensions": "60W x 30D x 29H inches"
    },
    {
        "id": "chair-001",
        "name": "Ergonomic Mesh Office Chair",
        "description": "Premium ergonomic office chair with breathable mesh back, "
                      "adjustable lumbar support, and 4D armrests. Features synchronized "
                      "tilt mechanism and memory foam seat cushion. Ideal for long work hours.",
        "category": "Office Chairs",
        "price": 299.99,
        "material": "Mesh, Metal, Premium Foam",
        "dimensions": "26W x 26D x 48H inches"
    },
    {
        "id": "sofa-001",
        "name": "Contemporary Sectional Sofa",
        "description": "Modern L-shaped sectional with chaise lounge. Upholstered in premium "
                      "performance fabric. Features deep seats, plush cushions, and solid "
                      "wood legs. Perfect for modern living rooms.",
        "category": "Sofas",
        "price": 1299.99,
        "material": "Performance Fabric, Solid Wood",
        "dimensions": "112W x 65D x 34H inches"
    },
    {
        "id": "table-001",
        "name": "Rustic Dining Table",
        "description": "Farmhouse-style dining table with solid wood construction. "
                      "Features distressed finish and trestle base. Seats 6-8 people "
                      "comfortably. Perfect for family gatherings.",
        "category": "Dining Tables",
        "price": 899.99,
        "material": "Solid Pine Wood",
        "dimensions": "72W x 42D x 30H inches"
    },
    {
        "id": "bed-001",
        "name": "Platform Storage Bed",
        "description": "Modern queen platform bed with integrated storage drawers. "
                      "Features upholstered headboard and durable wood slat support. "
                      "No box spring needed. Perfect for maximizing bedroom space.",
        "category": "Beds",
        "price": 799.99,
        "material": "Engineered Wood, Linen Fabric",
        "dimensions": "65W x 86D x 48H inches"
    }
]
print(f"""✓ Created PRODUCTS_DATA with {len(PRODUCTS_DATA)} records""")

✓ Created PRODUCTS_DATA with 5 records


## Importing Pipeline Components

We import the following for configuring our embedding ingestion pipeline:
- `Chunk`, the structured input for generating and ingesting embeddings
- `AlloyDBConnectionConfig` for configuring database connection information
- `AlloyDBVectorWriterConfig` for configuring write behavior like schema mapping and conflict resolution

In [ ]:
# Embedding-specific imports
from apache_beam.ml.rag.ingestion.alloydb import (
    AlloyDBVectorWriterConfig,
    AlloyDBConnectionConfig
)
from apache_beam.ml.rag.ingestion.base import VectorDatabaseWriteTransform
from apache_beam.ml.rag.types import Chunk, Content
from apache_beam.ml.rag.embeddings.huggingface import HuggingfaceTextEmbeddings

# Apache Beam core
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.ml.transforms.base import MLTransform

# What's next?

This colab covers several use cases that you can explore based on your needs after completing the Setup and Prerequisites:

🔰 **New to vector embeddings?**
- [Start with Quick Start](#scrollTo=Quick_Start_Basic_Vector_Ingestion)
- Uses simple out-of-box schema
- Perfect for initial testing

🚀 **Need to scale to large datasets?**
- [Go to Run on Dataflow](#scrollTo=Quick_Start_Run_on_Dataflow)
- Learn how to execute the same pipeline at scale
- Fully managed
- Process large datasets efficiently

🎯 **Have a specific schema?**
- [Go to Custom Schema](#scrollTo=Custom_Schema_with_Column_Mapping)
- Learn to use different column names
- Map metadata to individual columns

🔄 **Need to update embeddings?**
- [Check out Updating Embeddings](#scrollTo=Update_Embeddings_and_Metadata_with_Conflict_Resolution)
- Handle conflicts
- Selective field updates

🔗 **Need to generate and Store Embeddings for Existing AlloyDB Data??**
- [See Database Integration](#scrollTo=Adding_Embeddings_to_Existing_Database_Records)
- Read data from your AlloyDB table.
- Generate embeddings for the relevant fields.
- Update your table (or a related table) with the generated embeddings.

🤖 **Want to use Google's AI models?**
- [Try Vertex AI Embeddings](#scrollTo=Generate_Embeddings_with_VertexAI_Text_Embeddings)
- Use Google's powerful embedding models
- Seamlessly integrate with other Google Cloud services

<a name="quickstart"></a>
# Quick Start: Basic Vector Ingestion

This section shows the simplest way to generate embeddings and store them in AlloyDB.

## Create table with default schema

Before running the pipeline, we need a table to store our embeddings:

In [ ]:
table_name = "default_product_embeddings"
table_schema = f"""
  id VARCHAR PRIMARY KEY,
  embedding VECTOR(384) NOT NULL,
  content text,
  metadata JSONB
"""
setup_alloydb_table(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)
test_alloydb_connection(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

Connected to AlloyDB successfully!
Creating pgvector extension...
Creating table...
Setup completed successfully!
✓ default_product_embeddings table exists
✓ pgvector extension is installed


## Configure Pipeline Components

Now define the components that control the pipeline behavior:

### Map products to Chunks
- Our data is ingested as product dictionaries
- Embedding generation and ingestion processes `Chunks`
- We convert each product dictionary to a `Chunk` to configure what text to embed and what to treat as metadata

In [ ]:
from typing import Dict, Any

# The create_chunk function converts our product dictionaries to Chunks.
# This doesn't split the text - it simply structures it in the format
# expected by the embedding pipeline components.
def create_chunk(product: Dict[str, Any]) -> Chunk:
    """Convert a product dictionary into a Chunk object.

       The pipeline components (MLTransform, VectorDatabaseWriteTransform)
       work with Chunk objects. This function:
       1. Extracts text we want to embed
       2. Preserves product data as metadata
       3. Creates a Chunk in the expected format

    Args:
        product: Dictionary containing product information

    Returns:
        Chunk: A Chunk object ready for embedding
    """
    return Chunk(
        content=Content(
            text=f"{product['name']}: {product['description']}"
        ), # The text that will be embedded
        id=product['id'],  # Use product ID as chunk ID
        metadata=product,  # Store all product info in metadata
    )

### Generate embeddings with HuggingFace

We use a local pre-trained Hugging Face model to create vector embeddings from the product descriptions.

In [ ]:
huggingface_embedder = HuggingfaceTextEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

### Write to AlloyDB

The default AlloyDBVectorWriterConfig maps Chunk fields to database columns as:

| Database Column | Chunk Field | Description |
|----------------|-------------|-------------|
| id             | chunk.id    | Unique identifier |
| embedding      | chunk.embedding.dense_embedding | Vector representation |
| content        | chunk.content.text | Text that was embedded |
| metadata       | chunk.metadata | Additional data as JSONB |

In [ ]:
alloydb_writer_config = AlloyDBVectorWriterConfig(
    connection_config = AlloyDBConnectionConfig(JDBC_URL, DB_USER, DB_PASSWORD),
    table_name=table_name
)

## Assemble and Run Pipeline

Now we can create our pipeline that:
1. Takes our product data
2. Converts each product to a Chunk
3. Generates embeddings for each Chunk
4. Stores everything in AlloyDB

In [ ]:
import tempfile

# Executing on DirectRunner (local execution)
with beam.Pipeline() as p:
    _ = (
            p
            | 'Create Products' >> beam.Create(PRODUCTS_DATA)
            | 'Convert to Chunks' >> beam.Map(create_chunk)
            | 'Generate Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
              .with_transform(huggingface_embedder)
            | 'Write to AlloyDB' >> VectorDatabaseWriteTransform(
                alloydb_writer_config
            )
        )

## Verify Embeddings
Let's check what was written to our AlloyDB table:

In [ ]:
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


Found 5 products:
--------------------------------------------------------------------------------
id: desk-001
embedding: [-0.001057815,0.060585838,0.06550077,-0.020135332,0.0063834586,-0.016612189,-0.0031698928,0.10957789,-0.0820077,0.087466136,0.05336146,0.018569656,0.057456993,0.029180786,-0.0739839,-0.010892165,0.07001466,-0.03374081,0.051436793,0.07567188,-0.13465184,-0.032239668,-0.04638205,-0.029914198,-0.009278446,0.06718533,-0.051145084,0.042603016,0.007903699,-0.041032363,-0.04231419,-0.01325918,0.071415,0.040334247,-0.01094771,0.003317118,0.08076268,-0.017956244,-0.010060241,-0.050612617,-0.06333365,-0.015355688,-0.016382031,0.04253677,-0.0060182833,0.033938967,-0.022733817,-0.13714638,-0.046747543,-0.053437427,0.03156534,0.0036609878,0.008148901,0.0004498263,-0.0139052775,-0.0072347587,0.003329244,0.02151876,-0.002358143,-0.04545856,0.06712152,-0.06171683,0.0034830095,0.03768045,0.029724924,0.046725973,-0.041152626,0.022954624,0.06727486,-0.047817953,-0.022311574,-0.00636

## Quick Start Summary

In this section, you learned how to:
- Convert product data to the Chunk format expected by embedding pipelines
- Generate embeddings using a HuggingFace model
- Configure and run a basic embedding ingestion pipeline
- Store embeddings and metadata in AlloyDB

This basic pattern forms the foundation for all the advanced use cases covered in the following sections.

# Quick Start: Run on Dataflow

This section demonstrates how to launch the Quick Start embedding pipeline on Google Cloud Dataflow from the colab. While previous examples used DirectRunner for local execution, Dataflow provides a fully managed, distributed execution environment that is:
- Scalable: Automatically scales to handle large datasets
- Fault-tolerant: Handles worker failures and ensures exactly-once processing
- Fully managed: No need to provision or manage infrastructure

For more in-depth documentation to package your pipeline into a python file and launch a DataFlow job from the command line see [Create Dataflow pipeline using Python](https://cloud.google.com/dataflow/docs/quickstarts/create-pipeline-python).

## Create the AlloyDB table with default schema

Before running the pipeline, we need a table to store our embeddings:

In [ ]:
table_name = "default_dataflow_product_embeddings"
table_schema = f"""
  id VARCHAR PRIMARY KEY,
  embedding VECTOR(384) NOT NULL,
  content text,
  metadata JSONB
"""
setup_alloydb_table(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)
test_alloydb_connection(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

Connected to AlloyDB successfully!
Creating pgvector extension...
Creating table...
Setup completed successfully!
✓ default_dataflow_product_embeddings table exists
✓ pgvector extension is installed


## Save our Pipeline to a python file

To launch our pipeline job on DataFlow, we
1. Add command line arguments for passing pipeline options like AlloyDB credentioals
2. Save our pipeline code to a local file `basic_ingestion_pipeline.py`

In [ ]:
file_content = """
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import argparse
import tempfile

from apache_beam.ml.transforms.base import MLTransform
from apache_beam.ml.rag.types import Chunk, Content
from apache_beam.ml.rag.ingestion.base import VectorDatabaseWriteTransform
from apache_beam.ml.rag.ingestion.alloydb import AlloyDBVectorWriterConfig, AlloyDBConnectionConfig
from apache_beam.ml.rag.embeddings.huggingface import HuggingfaceTextEmbeddings
from apache_beam.options.pipeline_options import SetupOptions

PRODUCTS_DATA = [
    {
        "id": "desk-001",
        "name": "Modern Minimalist Desk",
        "description": "Sleek minimalist desk with clean lines and a spacious work surface. "
                      "Features cable management system and sturdy steel frame. "
                      "Perfect for contemporary home offices and workspaces.",
        "category": "Desks",
        "price": 399.99,
        "material": "Engineered Wood, Steel",
        "dimensions": "60W x 30D x 29H inches"
    },
    {
        "id": "chair-001",
        "name": "Ergonomic Mesh Office Chair",
        "description": "Premium ergonomic office chair with breathable mesh back, "
                      "adjustable lumbar support, and 4D armrests. Features synchronized "
                      "tilt mechanism and memory foam seat cushion. Ideal for long work hours.",
        "category": "Office Chairs",
        "price": 299.99,
        "material": "Mesh, Metal, Premium Foam",
        "dimensions": "26W x 26D x 48H inches"
    }
]

def run(argv=None):
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--alloydb_host',
        required=True,
        help='AlloyDB host'
    )
    parser.add_argument(
        '--alloydb_database',
        default='postgres',
        help='AlloyDB database name'
    )
    parser.add_argument(
        '--alloydb_table',
        required=True,
        help='AlloyDB table name'
    )
    parser.add_argument(
        '--alloydb_username',
        required=True,
        help='AlloyDB user name'
    )
    parser.add_argument(
        '--alloydb_password',
        required=True,
        help='AlloyDB password'
    )
    known_args, pipeline_args = parser.parse_known_args(argv)

    pipeline_options = PipelineOptions(pipeline_args)
    pipeline_options.view_as(SetupOptions).save_main_session = True

    with beam.Pipeline(options=pipeline_options) as p:
        _ = (
              p
              | 'Create Products' >> beam.Create(PRODUCTS_DATA)
              | 'Convert to Chunks' >> beam.Map(lambda product: Chunk(
                    content=Content(
                        text=f"{product['name']}: {product['description']}"
                    ), # The text that will be embedded
                    id=product['id'],  # Use product ID as chunk ID
                    metadata=product,  # Store all product info in metadata
                )
              )
              | 'Generate Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
                .with_transform(
                    HuggingfaceTextEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
                )
              | 'Write to AlloyDB' >> VectorDatabaseWriteTransform(
                  AlloyDBVectorWriterConfig(
                      connection_config=AlloyDBConnectionConfig(
                        jdbc_url=f'jdbc:postgresql://{known_args.alloydb_host}/{known_args.alloydb_database}',
                        username=known_args.alloydb_username,
                        password=known_args.alloydb_password
                    ),
                    table_name=known_args.alloydb_table
                  )
              )
          )

if __name__ == '__main__':
    run()
"""

with open("basic_ingestion_pipeline.py", "w") as f:
    f.write(file_content)

## Authenticate with Google Cloud

To launch a pipeline on Google Cloud, authenticate this notebook. Replace `<PROJECT_ID` with your Google Cloud project ID

In [ ]:
PROJECT_ID = "" # @param {type:'string'}
import os
os.environ['PROJECT_ID'] = PROJECT_ID

In [ ]:
from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

## Configure the Pipeline options



To run the pipeline on DataFlow we need
- A gcs bucket for staging DataFlow files. Replace `<BUCKET_NAME>`: the name of a valid Google Cloud Storage bucket. Don't include a gs:// prefix or trailing slashes
- Optionally set the the Google Cloud region that you want to run Dataflow in. Replace `<REGION>` with the desired location
- AlloyDB [private IP](https://cloud.google.com/alloydb/docs/private-ip#:~:text=Using%20private%20IP%20addresses%20keeps,instance%20and%20potential%20attack%20surface.) address to which the Datflow worker VM's have access. There are multiple ways to [connect](https://cloud.google.com/alloydb/docs/connection-overview) to your AlloyDB instance from Datflow, including
  -    Setting up [Private services access](https://cloud.google.com/alloydb/docs/about-private-services-access)
  -    Setting up [Private service connect](https://cloud.google.com/alloydb/docs/about-private-service-connect)

In [ ]:
BUCKET_NAME = '' # @param {type:'string'}
REGION = 'us-central1' # @param {type:'string'}
os.environ['BUCKET_NAME'] = BUCKET_NAME
os.environ['REGION'] = REGION

# Save AlloyDB credentioals to environment variables
PRIVATE_ALLOYDB_HOST = '' # @param {type:'string'}
os.environ['PRIVATE_ALLOYDB_HOST'] = PRIVATE_ALLOYDB_HOST
os.environ['DATABASE_NAME'] = DB_NAME
os.environ['ALLOYDB_USER'] = DB_USER
os.environ['ALLOYDB_PASSWORD'] = DB_PASSWORD

NETWORK = '' # @param {type:'string'}
SUBNETWORK = '' # @param {type:'string'}
os.environ['NETWORK'] = NETWORK
os.environ['SUBNETWORK'] = SUBNETWORK

## Provide additional Python dependencies to be installed on Worker VM's

We are making use of the HuggingFace `sentence-transformers` package to generate embeddings. Since this package is not installed on Worker VM's by default, we create a requirements.txt file with the additional dependencies to be installed on worker VM's.

See [Managing Python Pipeline Dependencies](https://beam.apache.org/documentation/sdks/python-pipeline-dependencies/) for more details.



In [ ]:
!echo "sentence-transformers" > ./requirements.txt
!cat ./requirements.txt

sentence-transformers


## Run Pipeline on Dataflow

We launch the pipeline via the command line, passing
- AlloyDB pipeline arguments defined in `basic_ingestion_pipeline.py`
- GCP Project ID
- Job Region
- The runner (DataflowRunner)
- Temp and Staging GCS locations for Pipeline artifacts
- Requirement file location for additional dependencies
- The VPC network and Subnetwork that has access to the AlloyDB instance

Once the job is launched, you can monitor its progress in the Google Cloud Console:
1. Go to https://console.cloud.google.com/dataflow/jobs
2. Select your project
3. Click on the job named "alloydb-dataflow-basic-embedding-ingest"
4. View detailed execution graphs, logs, and metrics

In [ ]:
!python ./basic_ingestion_pipeline.py \
  --project=$PROJECT_ID \
  --alloydb_username=$ALLOYDB_USER \
  --alloydb_host=${PRIVATE_ALLOYDB_HOST}:5432 \
  --alloydb_password=$ALLOYDB_PASSWORD \
  --alloydb_table=default_dataflow_product_embeddings \
  --alloydb_database=$DATABASE_NAME \
  --job_name=alloydb-dataflow-basic-embedding-ingest \
  --region=$REGION \
  --runner=DataflowRunner \
  --temp_location=gs://${BUCKET_NAME}/temp \
  --staging_location=gs://${BUCKEBUCKET_NAME}/staging \
  --requirements_file=requirements.txt \
  --network ${NETWORK} \
  --subnetwork regions/${REGION}/subnetworks/${SUBNETWORK}

## Verify the Written Embeddings

Let's check what was written to our AlloyDB table:

In [ ]:
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)



Found 2 products:
--------------------------------------------------------------------------------
id: chair-001
embedding: [0.02403435,0.04964332,0.023801215,0.023434723,-0.011474275,0.054758757,0.0066280337,-0.051804464,0.08406079,0.065616906,0.03321514,0.041509267,0.04673618,-0.008781659,0.052692704,-0.007216768,0.07655436,0.016920764,0.04152955,0.101700984,-0.046693634,0.0077436273,-0.021965409,0.020911854,0.0016925251,-0.0042127846,0.029753385,-0.009085065,-0.034632042,-0.0674854,0.00025547922,-0.05542992,0.0758605,-0.002696255,-0.047679216,0.003685547,0.023553496,-0.034157697,-0.060041208,0.012130825,-0.10083735,0.031097688,0.035963416,-0.04431593,-0.022646144,-0.017285833,-0.045440417,0.0034619803,-0.037636943,0.06251876,0.04322969,-0.055833027,-0.029337522,-0.05321623,-0.0012940895,0.010824212,-0.039729264,0.012684213,-0.04650574,-0.059219852,0.08073491,-0.060871,0.016936727,0.0326658,-0.09365397,0.02095731,-0.06390675,-0.07115554,-0.05761697,-0.03194697,-0.0983739,-0.02098407

# Advanced Use Cases

This section demonstrates more complex scenarios for using AlloyDB with Apache Beam for vector embeddings.

🎯 **Have a specific schema?**
- [Go to Custom Schema](#scrollTo=Custom_Schema_with_Column_Mapping)
- Learn to use different column names and transform values
- Map metadata to individual columns

🔄 **Need to update embeddings?**
- [Check out Updating Embeddings](#scrollTo=Update_Embeddings_and_Metadata_with_Conflict_Resolution)
- Handle conflicts
- Selective field updates

🔗 **Need to generate and Store Embeddings for Existing AlloyDB Data??**
- [See Database Integration](#scrollTo=Adding_Embeddings_to_Existing_Database_Records)
- Read data from your AlloyDB table.
- Generate embeddings for the relevant fields.
- Update your table (or a related table) with the generated embeddings.

🤖 **Want to use Google's AI models?**
- [Try Vertex AI Embeddings](#scrollTo=Generate_Embeddings_with_VertexAI_Text_Embeddings)
- Use Google's powerful embedding models
- Seamlessly integrate with other Google Cloud services


## Custom Schema with Column Mapping <a name="custom-schema"></a>

In this example, we'll create a custom schema that:
- Uses different column names
- Maps metadata to individual columns
- Uses functions to transform values

### ColumnSpec and ColumnSpecsBuilder


ColumnSpec specifies how to map data to a database column. For example:
```python
ColumnSpec(
    column_name="price",          # Database column
    python_type=float,            # Python Type for the value
    value_fn=lambda c: c.metadata['price'],  # Extract price from Chunk metadata to get actual value
    sql_typecast="::decimal"      # Optional SQL cast
)
```
creates an INSERT statement like:
```sql
INSERT INTO table (price) VALUES (?::decimal)
```
where the `?` placeholder is poulated with the value from our ingested data.

`ColumnSpecsBuilder` provides a builder and convenience methods to create these `ColumnSpecs`:

1. Core Field Mapping
   - `with_id_spec()` => Insert chunk.id as text in "id" column
   - `with_embedding_spec()` => Insert chunk.embedding as `float[]` in "embedding" column
   - `with_content_spec()` => Insert `chunk.content`.text as text in "content" column

   Note: All `with_id_spec`, `with_embedding_spec`, etc. methods allow overriding `column_name`, `python_type`, and `value_fn`.

2. Metadata Extraction
   - `add_metadata_field`: Creates a column from a `chunk.metadata` field
   - Handles type conversion based on specified SQL type

3. Custom Fields
   - `add_custom_column_spec`: Grants complete control over mapping `Chunk` data to database rows using `ColumnSpec`

Now, lets the table to store our embeddings:

### Create Custom Schema Table

In [ ]:
table_name = "custom_product_embeddings"
table_schema = """
    product_id VARCHAR PRIMARY KEY,
    vector_embedding VECTOR(384) NOT NULL,
    product_name VARCHAR,
    description TEXT,
    price DECIMAL,
    category VARCHAR,
    display_text VARCHAR,
    model_name VARCHAR,
    created_at TIMESTAMP
"""
setup_alloydb_table(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)
test_alloydb_connection(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

Connected to AlloyDB successfully!
Creating pgvector extension...
Creating table...
Setup completed successfully!
✓ custom_product_embeddings table exists
✓ pgvector extension is installed


### Configure Pipeline Components

#### Write to custom schema using ColumnSpecsBuilder


We configure ConlumnSpecsBuilder to map data as:

| Database Column | Chunk Field                               |
|-----------------|-------------------------------------------|
| `product_id`    | `chunk.id`                                |
| `vector_embedding`| `chunk.embedding.dense_embedding`        |
| `description`   | `chunk.content.text`                      |
| `product_name`  | `chunk.metadata['name']`                  |
| `price`         | `chunk.metadata['price']`                 |
| `category`      | `chunk.metadata['category']`              |
| `display_text`  | *Function that combines product name and price* |
| `model_name`    | *Function that returns the model name: "all-MiniLM-L6-v2"* |
| `created_at`    | *Function that returns the current timestamp cast to a SQL timestamp* |

In [ ]:
from apache_beam.ml.rag.ingestion.alloydb import ColumnSpec
from apache_beam.ml.rag.ingestion.alloydb import ColumnSpecsBuilder
from datetime import datetime

column_specs = (
    ColumnSpecsBuilder()
    # Write chunk.id to a column named "product_id"
    .with_id_spec(column_name='product_id')
    # Write chunk.embedding.dense_embedding to a column named "vector_embedding"
    .with_embedding_spec(column_name='vector_embedding')
    # Write chunk.content.text to a column named "description"
    .with_content_spec(column_name='description')
    # Write chunk.metadata.['product_name'] to a column named "product_name"
    .add_metadata_field(
        field='name',
        column_name='product_name',
        python_type=str
    )
    # Write chunk.metadata.['price'] to a column named "price"
    .add_metadata_field(
        field='price',
        column_name='price',
        python_type=float
    )
    # Write chunk.metadata.['category'] to a column named "category"
    .add_metadata_field(
        field='category',
        column_name='category',
        python_type=str
    )
    # Write custom field using value_fn to column named "display_text" using
    # ColumnSpec.text convenience method
    .add_custom_column_spec(
        ColumnSpec.text(
          column_name='display_text',
        value_fn=lambda chunk: \
          f"{chunk.metadata['name']} - ${chunk.metadata['price']:.2f}"
        )
    )
    # Store model used to generate embedding using ColumnSpec constructor
    .add_custom_column_spec(
        ColumnSpec(
          column_name='model_name',
          python_type=str,
          value_fn=lambda _: "all-MiniLM-L6-v2"
        )
    )
    .add_custom_column_spec(
        ColumnSpec(
          column_name='created_at',
          python_type=str,
          value_fn=lambda _: datetime.now().isoformat(),
          sql_typecast="::timestamp"
        )
    )
    .build()
)

### Assemble and Run Pipeline

Now we can create our pipeline that will:
1. Take our product data
2. Convert each product to a Chunk
3. Generate embeddings for each Chunk
4. Store everything in AlloyDB with our custom schema configuration

In [ ]:
import tempfile # For storing MLTransform artifacts

# Executing on DirectRunner (local execution)
with beam.Pipeline() as p:
    _ = (
            p
            | 'Create Products' >> beam.Create(PRODUCTS_DATA)
            | 'Convert to Chunks' >> beam.Map(lambda product: Chunk(
                    content=Content(
                        text=f"{product['name']}: {product['description']}"
                    ), # The text that will be embedded
                    id=product['id'],  # Use product ID as chunk ID
                    metadata=product,  # Store all product info in metadata
                )
              )
            | 'Generate Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
              .with_transform(HuggingfaceTextEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
            | 'Write to AlloyDB' >> VectorDatabaseWriteTransform(
                AlloyDBVectorWriterConfig(
                    connection_config = AlloyDBConnectionConfig(JDBC_URL, DB_USER, DB_PASSWORD),
                    table_name=table_name,
                    column_specs=column_specs
                )
            )
        )

### Verify the Written Embeddings

Let's check what was written to our AlloyDB table:

In [ ]:
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


Found 5 products:
--------------------------------------------------------------------------------
product_id: desk-001
vector_embedding: [-0.001057815,0.060585838,0.06550077,-0.020135332,0.0063834586,-0.016612189,-0.0031698928,0.10957789,-0.0820077,0.087466136,0.05336146,0.018569656,0.057456993,0.029180786,-0.0739839,-0.010892165,0.07001466,-0.03374081,0.051436793,0.07567188,-0.13465184,-0.032239668,-0.04638205,-0.029914198,-0.009278446,0.06718533,-0.051145084,0.042603016,0.007903699,-0.041032363,-0.04231419,-0.01325918,0.071415,0.040334247,-0.01094771,0.003317118,0.08076268,-0.017956244,-0.010060241,-0.050612617,-0.06333365,-0.015355688,-0.016382031,0.04253677,-0.0060182833,0.033938967,-0.022733817,-0.13714638,-0.046747543,-0.053437427,0.03156534,0.0036609878,0.008148901,0.0004498263,-0.0139052775,-0.0072347587,0.003329244,0.02151876,-0.002358143,-0.04545856,0.06712152,-0.06171683,0.0034830095,0.03768045,0.029724924,0.046725973,-0.041152626,0.022954624,0.06727486,-0.047817953,-0.022

## Update Embeddings and Metadata with Conflict Resolution <a name="updating"></a>

This section demonstrates how to handle periodic updates to product descriptions and their embeddings using the default schema. We'll show how embeddings and metadata get updated when product descriptions change.


### Create table with desired schema

Let's use the same default schema as in Quick Start:

In [ ]:
table_name = "mutable_product_embeddings"
table_schema = f"""
  id VARCHAR PRIMARY KEY,
  embedding VECTOR(384) NOT NULL,
  content text,
  metadata JSONB,
  created_at TIMESTAMP NOT NULL DEFAULT NOW()
"""
setup_alloydb_table(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)
test_alloydb_connection(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

Connected to AlloyDB successfully!
Creating pgvector extension...
Creating table...
Setup completed successfully!
✓ mutable_product_embeddings table exists
✓ pgvector extension is installed


### Sample Data: Day 1 vs Day 2

In [ ]:
PRODUCTS_DATA_DAY1 = [
    {
        "id": "desk-001",
        "name": "Modern Minimalist Desk",
        "description": "Sleek minimalist desk with clean lines and a spacious work surface. "
                      "Features cable management system and sturdy steel frame.",
        "category": "Desks",
        "price": 399.99,
        "update_timestamp": "2024-02-18"
    }
]

PRODUCTS_DATA_DAY2 = [
    {
        "id": "desk-001",  # Same ID as Day 1
        "name": "Modern Minimalist Desk",
        "description": "Updated: Sleek minimalist desk with built-in wireless charging. "
                      "Features cable management system, sturdy steel frame, and Qi charging pad. "
                      "Perfect for modern tech-enabled workspaces.",
        "category": "Smart Desks",  # Category changed
        "price": 449.99,  # Price increased
        "update_timestamp": "2024-02-19"
    }
]

### Configure Pipeline Components

#### Writer with Conflict Resolution

In [ ]:
from apache_beam.ml.rag.ingestion.alloydb import (
    AlloyDBVectorWriterConfig,
    AlloyDBConnectionConfig,
    ConflictResolution
)

# Define how to handle conflicts - update all fields when ID matches
conflict_resolution = ConflictResolution(
    on_conflict_fields="id",  # Identify records by ID
    action="UPDATE",         # Update existing records
    update_fields=["embedding", "content", "metadata"]
)

# Create writer config with conflict resolution
alloydb_writer_config = AlloyDBVectorWriterConfig(
    connection_config=AlloyDBConnectionConfig(JDBC_URL, DB_USER, DB_PASSWORD),
    table_name=table_name,
    conflict_resolution=conflict_resolution,
)

In [ ]:
huggingface_embedder = HuggingfaceTextEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

### Run Day 1 Pipeline

First, let's ingest our initial product data:

In [ ]:
# Executing on DirectRunner (local execution)
with beam.Pipeline() as p:
    _ = (
        p
        | 'Create Day 1 Products' >> beam.Create(PRODUCTS_DATA_DAY1)
        | 'Convert Day 1 to Chunks' >> beam.Map(lambda product: Chunk(
                content=Content(
                    text=f"{product['name']}: {product['description']}"
                ), # The text that will be embedded
                id=product['id'],  # Use product ID as chunk ID
                metadata=product,  # Store all product info in metadata
            )
          )
        | 'Generate Day1 Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
          .with_transform(HuggingfaceTextEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
        | 'Write Day 1 to AlloyDB' >> VectorDatabaseWriteTransform(
            alloydb_writer_config
        )
    )

#### Verify Initial Data

In [ ]:
print("\nAfter Day 1 ingestion:")
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


After Day 1 ingestion:

Found 1 products:
--------------------------------------------------------------------------------
id: desk-001
embedding: [-0.0045002503,0.05900373,0.07442122,-0.0151996,0.006877708,-0.010805947,0.0040495754,0.11678752,-0.07902289,0.08912011,0.067379504,0.014676305,0.060260665,0.027606748,-0.07220319,-0.017484626,0.072982155,-0.040194146,0.046230145,0.050661087,-0.12877946,-0.047799468,-0.041656304,-0.02108199,-0.010726097,0.06844871,-0.049604487,0.043336596,0.0064744623,-0.06115,-0.04501372,-0.02326481,0.06976917,0.03558554,-0.00513683,-0.00085037545,0.08450821,-0.024197027,-0.032184083,-0.058323413,-0.074653424,-0.010908077,-0.024047483,0.046621956,-0.0043276376,0.020488564,-0.0045746,-0.13073471,-0.050565727,-0.050982136,0.02560689,-0.015003142,-0.002509348,0.0008339738,0.0016991752,-0.006976142,0.008729271,0.019776281,-0.0039278003,-0.03742233,0.0570335,-0.06417092,-0.0076810643,0.04203856,0.018476766,0.044952694,-0.040934063,0.019591931,0.050816394,-0.045

### Run Day 2 Pipeline

Now let's process our updated product data:

In [ ]:
# Executing on DirectRunner (local execution)
with beam.Pipeline() as p:
    _ = (
        p
        | 'Create Day 2 Products' >> beam.Create(PRODUCTS_DATA_DAY2)
        | 'Convert Day 2 to Chunks' >> beam.Map(lambda product: Chunk(
                    content=Content(
                        text=f"{product['name']}: {product['description']}"
                    ), # The text that will be embedded
                    id=product['id'],  # Use product ID as chunk ID
                    metadata=product,  # Store all product info in metadata
                )
              )
        | 'Generate Day 2 Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
          .with_transform(HuggingfaceTextEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
        | 'Write Day 2 to AlloyDB' >> VectorDatabaseWriteTransform(
            alloydb_writer_config
        )
    )

#### Verify Updated Data

In [ ]:
print("\nAfter Day 2 ingestion:")
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


After Day 2 ingestion:

Found 1 products:
--------------------------------------------------------------------------------
id: desk-001
embedding: [-0.069688216,0.09990969,0.054291457,0.021926325,0.026119737,-0.009320738,0.04093818,0.08397181,-0.08460739,0.08246407,0.13822316,-0.0033618943,0.084282085,0.047409005,-0.028424418,-0.018919641,0.07962887,-0.054636743,0.07036568,0.03590471,-0.13375837,-0.007273606,0.010672821,-0.059215046,0.045872547,0.06291143,-0.016371163,0.019913405,-0.021351157,-0.041431487,-0.014848063,-0.01840748,0.071445964,-0.008601728,-0.032404777,-0.00023905447,0.08869474,-0.013788823,-0.0038640038,-0.056946076,-0.092713885,-0.01890048,0.005022845,0.055990435,0.009209997,0.030351037,-0.0015497233,-0.09739816,-0.05679334,-0.050131287,0.08160355,-0.058753196,0.027757475,0.0313377,-0.026671894,-0.030177608,-0.010398384,0.033542562,0.030896632,-0.08753507,0.037424564,-0.052436404,0.020135561,0.06772314,0.010598365,0.03482641,-0.031074163,0.043301966,0.033493366,-0.050

### What Changed?

Key points to notice:

1. The embedding vector changed because the product description was updated
2. The metadata JSONB field contains the updated category, price, and timestamp
3. The content field reflects the new description
4. The original ID remained the same

This pattern allows you to:
- Update embeddings when source text changes
- Maintain referential integrity with consistent IDs
- Track changes through the metadata field
- Handle conflicts gracefully using AlloyDB's conflict resolution


## Adding Embeddings to Existing Database Records <a name="integration" target="_blank"></a>

This section demonstrates how to:
1. Read existing product data from a database
2. Generate embeddings for that data
3. Write the embeddings back to the database

In [ ]:
table_name = "existing_products"
table_schema = """
    id VARCHAR PRIMARY KEY,
    title VARCHAR NOT NULL,
    description TEXT,
    price DECIMAL,
    embedding VECTOR(384)
"""

In [ ]:
#@title Postgres helpers for inserting initial records
def setup_initial_data(host: str,
                      database: str,
                      table_name: str,
                      user: str,
                      password: str,
                      port: int = 5432):
    """Set up table and insert sample product data."""
    conn_string = f"host={host} dbname={database} user={user} password={password} port={port}"

    try:
        conn = psycopg2.connect(conn_string)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = conn.cursor()

        # Create pgvector extension
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

        # Create products table
        cur.execute(f"DROP TABLE IF EXISTS {table_name};")
        cur.execute(f"CREATE TABLE {table_name} ({table_schema});")

        # Insert sample data
        sample_products = [
            (
                "lamp-001",
                "Artisan Table Lamp",
                "Hand-crafted ceramic table lamp with linen shade. "
                "Features dimmable LED bulb and touch-sensitive base.",
                129.99
            ),
            (
                "mirror-001",
                "Floating Wall Mirror",
                "Modern circular mirror with hidden mounting system. "
                "Perfect for entryways and contemporary spaces.",
                199.99
            ),
            (
                "vase-001",
                "Contemporary Ceramic Vase",
                "Minimalist vase with matte finish. "
                "Ideal for dried or fresh flower arrangements.",
                79.99
            )
        ]

        cur.executemany(
            f"""
            INSERT INTO {table_name} (id, title, description, price)
            VALUES (%s, %s, %s, %s)
            """,
            sample_products
        )

        print("✓ Sample products inserted successfully")

    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()

In [ ]:
setup_initial_data(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

✓ Sample products inserted successfully


### Read from Database and Generate Embeddings

Now let's create a pipeline to read the existing data, generate embeddings, and write back:

In [ ]:
from apache_beam.io.jdbc import ReadFromJdbc
from apache_beam.io.jdbc import WriteToJdbc
from apache_beam.ml.rag.ingestion.alloydb import ColumnSpecsBuilder

# Configure database writer
alloydb_writer_config = AlloyDBVectorWriterConfig(
    connection_config=AlloyDBConnectionConfig(JDBC_URL, DB_USER, DB_PASSWORD),
    table_name=table_name,
    column_specs=(
        ColumnSpecsBuilder()
          .with_id_spec()
          .with_embedding_spec()
          # Add a placeholder value for the title column, because it has a
          # NOT NULL constraint. Insert with Conflict resolution statements in
          # Postgres requires all NOT NULL fields to have a value, even if the
          # value will not be updated (the original title is preserved).
          .add_custom_column_spec(
            ColumnSpec.text("title", value_fn=lambda x: "")
           )
          .build()
    ),
    conflict_resolution=ConflictResolution(
        on_conflict_fields="id",
        action="UPDATE",
        update_fields=["embedding"]  # Update the embedding field
    )
)

# Create and run pipeline  on DirectRunner (local execution)
with beam.Pipeline() as p:
    # Read existing products
    rows = (
        p
        | "Read Products" >> ReadFromJdbc(
            table_name=table_name,
            driver_class_name="org.postgresql.Driver",
            jdbc_url=JDBC_URL,
            username=DB_USER,
            password=DB_PASSWORD,
            query=f"SELECT id, title, description FROM {table_name}"
        )
    )

    # Generate and write embeddings
    _ = (
        rows
        | "Convert to Chunks" >> beam.Map(lambda row: Chunk(
              id=row.id,
              content=Content(text=f"{row.title}: {row.description}")
            )
          )
        | "Generate Embeddings" >> MLTransform(
            write_artifact_location=tempfile.mkdtemp()
        ).with_transform(HuggingfaceTextEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
        | "Write Back to AlloyDB" >> VectorDatabaseWriteTransform(
            alloydb_writer_config
        )
    )



### Verify Data

In [ ]:
print("\nAfter embedding generation:")
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


After embedding generation:

Found 3 products:
--------------------------------------------------------------------------------
id: lamp-001
title: Artisan Table Lamp
description: Hand-crafted ceramic table lamp with linen shade. Features dimmable LED bulb and touch-sensitive base.
price: 129.99
embedding: [0.031689487,0.015019406,-0.027992113,0.011794099,0.027890578,0.012311332,0.05094233,0.039530788,0.0062339273,0.09796203,0.036408335,-0.045579962,-0.048446417,0.030952323,-0.01628782,0.0050955242,-0.00046880535,-0.024875749,0.03709692,0.089265496,0.05788519,-0.03168925,-0.027708465,-0.016526584,0.014046952,0.019951709,0.024461117,0.039271634,-0.039026197,-0.05182474,-0.058577348,0.047831777,-0.040588457,0.083615184,-0.09252382,-0.031066956,-0.048101336,-0.052257475,0.013567206,0.0101343775,-0.03135944,-0.0930071,0.018740054,0.024566485,0.050544124,-0.07123016,0.05440473,-0.0501441,-0.14550053,-0.06971258,0.025340369,0.02691406,0.09685166,0.018577438,0.010204357,0.03331682,-0.0914395

What Happened?
1. We started with a table containing product data but no embeddings
2. Read the existing records using ReadFromJdbc
3. Converted rows to Chunks, combining title and description for embedding
4. Generated embeddings using our model
5. Wrote back to the same table, updating only the embedding field
Preserved all other fields (price, etc.)

This pattern is useful when:

- You have an existing product database
- You want to add embeddings without disrupting current data
- You need to maintain existing schema and relationships


## Generate Embeddings with VertexAI Text Embeddings

This section demonstrates how to use use the Vertex AI text-embeddings API to generate text embeddings that use Googles large generative artificial intelligence (AI) models.

Vertex AI models are subject to [Rate Limits and Quotas](https://cloud.google.com/vertex-ai/generative-ai/docs/quotas#view-the-quotas-by-region-and-by-model) and Dataflow automatically retries throttled requests with exponential backoff.


For more information, see [Get text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) in the Vertex AI documentation.

### Authenticate with Google Cloud
To use the Vertex AI API, we authenticate with Google Cloud.

In [ ]:
# Replace <PROJECT_ID> with a valid Google Cloud project ID.
PROJECT_ID = '' # @param {type:'string'}

from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

### Create AlloyDB table with default schema

First we create a table to store our embeddings:

In [ ]:
table_name = "vertex_product_embeddings"
table_schema = f"""
  id VARCHAR PRIMARY KEY,
  embedding VECTOR(768) NOT NULL,
  content text,
  metadata JSONB
"""
setup_alloydb_table(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)
test_alloydb_connection(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)

Connected to AlloyDB successfully!
Creating pgvector extension...
Creating table...
Setup completed successfully!
✓ vertex_product_embeddings table exists
✓ pgvector extension is installed


### Configure Embedding Handler

Import the `VertexAITextEmbeddings` handler, and specify the desired `textembedding-gecko` model.

In [ ]:
from apache_beam.ml.rag.embeddings.vertex_ai import VertexAITextEmbeddings

vertexai_embedder = VertexAITextEmbeddings(model_name="textembedding-gecko@latest")

### Run the Pipeline

In [ ]:
import tempfile

# Executing on DirectRunner (local execution)
with beam.Pipeline() as p:
    _ = (
            p
            | 'Create Products' >> beam.Create(PRODUCTS_DATA)
            | 'Convert to Chunks' >> beam.Map(lambda product: Chunk(
                    content=Content(
                        text=f"{product['name']}: {product['description']}"
                    ), # The text that will be embedded
                    id=product['id'],  # Use product ID as chunk ID
                    metadata=product,  # Store all product info in metadata
                )
              )
              | 'Generate Embeddings' >> MLTransform(write_artifact_location=tempfile.mkdtemp())
                .with_transform(
                    vertexai_embedder
                )
              | 'Write to AlloyDB' >> VectorDatabaseWriteTransform(
                  AlloyDBVectorWriterConfig(
                      connection_config=AlloyDBConnectionConfig(
                        jdbc_url=JDBC_URL,
                        username=DB_USER,
                        password=DB_PASSWORD
                    ),
                    table_name=table_name
                  )
              )
          )

### Verify Embeddings

In [ ]:
print("\nAfter embedding generation:")
verify_embeddings(ALLOYDB_HOST, DB_NAME, table_name, DB_USER, DB_PASSWORD)


After embedding generation:

Found 5 products:
--------------------------------------------------------------------------------
id: desk-001
embedding: [0.039224163,-0.0330951,-0.025687244,0.013073655,0.08133947,0.04555349,0.012126796,-0.035832036,0.024597123,0.027825661,-0.011793963,-0.014143333,-0.035762396,0.009388906,-0.023734972,-0.052758113,0.02155404,0.041994106,-0.026714081,0.03136702,0.049227387,0.036142185,0.008596038,0.0018139591,0.014294273,-0.04681292,0.023720035,-0.033541713,-0.020142222,-0.005330984,-0.01809484,0.03847957,-0.054286744,0.04666274,0.018944228,-0.024427071,0.0036434168,0.027388662,-0.026692593,0.0099747535,0.037849277,-0.05586371,-0.0328882,0.033327486,-0.0141837625,0.019386668,-0.03283112,0.0004936014,0.028712625,-0.039343365,0.0050554713,-0.021856239,0.0350442,0.01740936,-0.030664355,-0.039946113,0.058474638,-0.02889143,-0.019847749,0.009937114,0.051981255,0.04023026,0.049771704,0.003612052,0.037882626,-0.052851994,-0.03170944,0.049247973,0.07078638,0.01